In [ ]:
"""
THIS SCRIPT IS DESIGNED TO RUN ON THE NCI SUPERCOMPUTER.

You'll need to use your own sounding datasets to build the required profile information
"""

In [2]:
from glob import glob
from datetime import datetime
import xarray as xr
import numpy as np
from scipy.interpolate import interp1d

In [8]:
def _sounding_interp(snd_temp, snd_height, target_temp):
    """
    Provides an linear interpolated height for a target temperature using a
    sounding vertical profile. Looks for first instance of temperature
    below target_temp from surface upward.

    Parameters
    ----------
    snd_temp : ndarray
        Temperature data (degrees C).
    snd_height : ndarray
        Relative height data (m).
    target_temp : float
        Target temperature to find height at (m).

    Returns
    -------
    intp_h: float
        Interpolated height of target_temp (m).

    """
    intp_h = np.nan

    #check if target_temp is warmer than lowest level in sounding
    if target_temp>snd_temp[0]:
        print('warning, target temp level below sounding, returning ground level (0m)')
        return 0.
    
    # find index above and below freezing level
    mask = np.where(snd_temp < target_temp)
    above_ind = mask[0][0]

    # index below
    below_ind = above_ind - 1
    
    # apply linear interplation to points above and below target_temp
    set_interp = interp1d(
        snd_temp[below_ind:above_ind+1],
        snd_height[below_ind:above_ind+1], kind='linear')
    
    # apply interpolant
    intp_h = set_interp(target_temp)
    
    return intp_h


def mesh_levels(request_dt, request_lat, request_lon):
    
    #set era path
    era5_root = '/g/data/ub4/era5/netcdf/pressure'
    
    #build file paths
    month_str = request_dt.month
    year_str = request_dt.year
    temp_ffn = glob(f'{era5_root}/t/{year_str}/t_era5_aus_{year_str}{month_str:02}*.nc')[0]
    geop_ffn = glob(f'{era5_root}/z/{year_str}/z_era5_aus_{year_str}{month_str:02}*.nc')[0]
    rh_ffn   = glob(f'{era5_root}/r/{year_str}/r_era5_aus_{year_str}{month_str:02}*.nc')[0]
    
    #extract data
    with xr.open_dataset(temp_ffn) as temp_ds:
        temp_data = temp_ds.t.sel(longitude=request_lon, method='nearest').sel(latitude=request_lat, method='nearest').sel(time=request_dt, method='nearest').data[:] - 273.15 #units: deg K -> C
    with xr.open_dataset(geop_ffn) as geop_ds:
        geop_data = geop_ds.z.sel(longitude=request_lon, method='nearest').sel(latitude=request_lat, method='nearest').sel(time=request_dt, method='nearest').data[:]/9.80665 #units: m**2 s**-2 -> m
    with xr.open_dataset(rh_ffn) as rh_ds:
        rh_data = rh_ds.r.sel(longitude=request_lon, method='nearest').sel(latitude=request_lat, method='nearest').sel(time=request_dt, method='nearest').data[:] #units: percentage        
    #flipdata (ground is first row)
    temp_data = np.flipud(temp_data)
    geop_data = np.flipud(geop_data)
    rh_data = np.flipud(rh_data)
    
    #interpolate to 0C and -20C levels
    fz_level = np.round(_sounding_interp(temp_data, geop_data, 0))
    minus_20_level = np.round(_sounding_interp(temp_data, geop_data, -20))
    
    return [temp_data, geop_data, rh_data, fz_level, minus_20_level]

In [10]:
request_dt = datetime.strptime('20200119_0500', '%Y%m%d_%H%M')
request_lat = -37.83
request_lon = 145.05
[temp_data, geop_data, rh_data, fz_level, minus_20_level] = mesh_levels(request_dt, request_lat, request_lon)

#save to npzfile
np.savez('example_data/melb_20200119_era5_sounding.npz', t=temp_data, z=geop_data, r=rh_data, 
         fz_level=fz_level, minus_20_level=minus_20_level)